<a href="https://colab.research.google.com/github/yanli499/ChatTime/blob/master/YLL_Facial_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import shutil
import copy
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from PIL import Image
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
from torch.autograd import Variable

In [0]:
# Mount our Google Drive
# re-run whenever needed
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# re-run when needed
# classes are folders in each directory with these names
classes = ['afraid','angry','disgusted','happy','neutral','sad','surprised']

# emotion label for KDEF photos
emotion_code = {"AF":"afraid", "AN":"angry", "DI":"disgusted", "HA":"happy", 
                "NE":"neutral", "SA":"sad", "SU":"surprised"}
emotion_enum = {"afraid":1, "angry":2, "disgusted":3, "happy":4, 
                "neutral":5, "sad":6, "surprised":7}

fileset = ['train', 'val', 'test']

data_dir='/content/drive/My Drive/Colab Notebooks/Faces'

In [0]:
data = []
target = []

for f in fileset:
    file_dir = data_dir + '/' + f
    for c in classes:
        filepath = file_dir + '/' + c
        print(filepath)
        for file in os.listdir(filepath):
            target.append(emotion_enum[c])
            im = Image.open(filepath + "/" + file)
            im = im.crop((16, 16, 240, 240))
            np_im = np.array(im)
            np_im = np.ravel(np_im)
            data.append(np_im)

/content/drive/My Drive/Colab Notebooks/Faces/train/afraid
/content/drive/My Drive/Colab Notebooks/Faces/train/angry
/content/drive/My Drive/Colab Notebooks/Faces/train/disgusted
/content/drive/My Drive/Colab Notebooks/Faces/train/happy
/content/drive/My Drive/Colab Notebooks/Faces/train/neutral
/content/drive/My Drive/Colab Notebooks/Faces/train/sad
/content/drive/My Drive/Colab Notebooks/Faces/train/surprised
/content/drive/My Drive/Colab Notebooks/Faces/val/afraid
/content/drive/My Drive/Colab Notebooks/Faces/val/angry
/content/drive/My Drive/Colab Notebooks/Faces/val/disgusted
/content/drive/My Drive/Colab Notebooks/Faces/val/happy
/content/drive/My Drive/Colab Notebooks/Faces/val/neutral
/content/drive/My Drive/Colab Notebooks/Faces/val/sad
/content/drive/My Drive/Colab Notebooks/Faces/val/surprised
/content/drive/My Drive/Colab Notebooks/Faces/test/afraid
/content/drive/My Drive/Colab Notebooks/Faces/test/angry
/content/drive/My Drive/Colab Notebooks/Faces/test/disgusted
/content

In [0]:
# importing necessary libraries 
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

# # loading the dataset
# faces = datasets.fetch_olivetti_faces()
# X = faces.data
# y = faces.target
X = np.array(data)
y = np.array(target)

print(X.shape)
print(y.shape)

# thus, need to create data w shape (7 * 140, 224 * 224) for total # of samples
# and each row = pixel of photo
# target = labels of all samples

# dividing X, y into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 

# training a Naive Bayes classifier 
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, y_train) 
gnb_predictions = gnb.predict(X_test) 

# accuracy on X_test 
accuracy = gnb.score(X_test, y_test) 
print(accuracy)

# creating a confusion matrix 
cm = confusion_matrix(y_test, gnb_predictions) 
print(cm)

(833, 50176)
(833,)
0.3157894736842105
[[17  3  0  6  4  0  3]
 [ 8  8  0  2  3  0  1]
 [ 8  4  0  7  2  1  2]
 [ 3  2  0 25  1  0  1]
 [10  0  0  9  5  0  6]
 [12  3  0  7  6  0  3]
 [15  6  0  2  3  0 11]]
